<a href="https://colab.research.google.com/github/manishkolla/B-Tree-Implementation-and-Automated-Data-Insertion/blob/main/B_Tree_Implementation_and_Automated_Data_Insertion_(CS4520).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Title: B-Tree Implementation and Automated Data Insertion
## CS 4520 Design & Analysis: Algorithms  by Prof. Shiraj Pokharel
### Project Done By:
1.   Manish Kolla
2.   Ashwin Gopalakrishnan

#### Input: Data File with index which needs to be converted to BTree
#### Output: Converted Btree from Dataset and a user interface to manipulate the Btree
##### Manipulations Include
1.   Searching
2.   Removal
3.   Depth First Search (Pre Order, In Order, Post Order)
4.   Breadth First Search (Level Order Traversal)
5.   Exiting

##### Responsibilities

###### Manish Kolla:
###### Coding: Searching and Insertion code of the Btree which include insert(), insert_non_full(), split_child(), Data File Parsing file(), Pre order, In order, Post Order Traversal, Breath First Search, Printing the BTree, debugging and trying to solve the challenges faced
###### Paper: Abstract, Material and Methods, Understanding and history of BTrees, Algorithm Design, IEEE Paper Formating


---


##### Ashwin Gopalakrishnan:
###### Coding: Basic Implementation, Deletion part of the code and balancing the Btree which include delete(),_delete_key_from_node(), _delete_from_child(), _get_predecessor(), _fix_deficiency(), _borrow_from_left_sibling(),_borrow_from_right_sibling(), _merge_with_left_sibling(), _merge_with_right_sibling(), debugging and trying to solve the challenges faced

###### Paper: Introduction, Algorithm Design, Critique, IEEE Paper Formating


---


Challenges Faced:
1. Duplication of elements when they are moving from child node to parent node in the split_child()
2. Errors in the _fix_deficiency() part of the code when the element is deleted from the leaf, the tree is becoming unbalanced
3. Errors in Search(), unable to identify the elements in the leaf accurately
4. Problems with the insert_non_null() when all the elements in the node are deleted
5. Trying it make it as user-friendly as possible for file()


# Final BTree Code


In [ ]:
class BTreeNode:
    def __init__(self, minDegree):
        self.keys = []
        self.children = []
        self.leaf = True
        self.t = minDegree  # The minimum degree (defines the range for the number of keys)
    def split_child(self, childIndex, child):
          # Splitting the child into two nodes and moving the part of the child to the parent and removing that from the child
          new_node = BTreeNode(child.t)
          new_node.leaf = child.leaf
          t = child.t

          # Give the new node the second half which is t-1 of child's keys and children
          new_node.keys = child.keys[t-1:]
          child.keys=child.keys[:t-1]
          #new_node.keys = child.keys[t - 1:]
          if not child.leaf:
              new_node.children = child.children[t:]
              child.children = child.children[:t]

          # Reduce the number of keys in child
          child.keys = child.keys[:t]

          # As child is split, its parent gets a new child
          self.children.insert(childIndex + 1, new_node)
          #self.keys.insert(childIndex, new_node.keys[-1])
          self.keys.insert(childIndex, new_node.keys.pop(0))
          #child.keys.pop(t-1)
    def insert_non_full(self, key):
      # Determining the insert position in the node's key list
        insertPosition = len(self.keys) - 1
        if self.leaf:
            # Insert the new key into the node's key list if its a leaf
            self.keys.append(key)
            self.keys.sort()
        else:
            # Find the child which is going to have the new key if its not the leaf
            while insertPosition >= 0 and key < self.keys[insertPosition]:
                insertPosition -= 1
            insertPosition += 1

            # If the child is full which is 2t-1, then split it before inserting
            if len(self.children[insertPosition].keys) == 2 * self.t - 1:
                self.split_child(insertPosition, self.children[insertPosition])
                if key > self.keys[insertPosition]:
                    insertPosition += 1
            #Recursively insert into the appropriate child
            self.children[insertPosition].insert_non_full(key)
from collections import deque
class BTree:
    def __init__(self, t):
        self.root = BTreeNode(t)
        self.t = t
    def insert(self, key):
        root = self.root
        if len(root.keys) == 2 * self.t - 1:
            # If root is full then call the split child and also B-tree grows in height
            new_node = BTreeNode(self.t)
            new_node.children.insert(0, root)
            new_node.leaf = False
            new_node.split_child(0, root)
            # New node is becoming the new root
            self.root = new_node
        self.root.insert_non_full(key)
    def search(self, searchKey):
        node = self.root
        while True:
            # Check current node keys for searchKey
            for i in range(len(node.keys)):
                if node.keys[i] == searchKey:
                    return True
            # Traversal logic if not found in the node
            if node.leaf:
                break
            i = len(node.keys) - 1
            while i >= 0 and searchKey < node.keys[i]:
                i -= 1
            i += 1
            # Going to the next child node
            node = node.children[i]

        return False
    def delete(self, key):
        if not self.root:
            return False

        # Call the delete function starting from the root
        self._delete(self.root, key)

    def _delete(self, node, key):
      #if the key is none
        if node is None:
            return
        # Find the index of the key to be deleted
        keyIndex = 0
        while keyIndex < len(node.keys) and key > node.keys[keyIndex]:
            keyIndex += 1

        # If the key is present in the current node
        if keyIndex < len(node.keys) and key == node.keys[keyIndex]:
            #print(f"Key {key} found in node {node.keys}")
            self._delete_key_from_node(node, key, keyIndex)
        else:
            # If the key is not present in the current node, determine the child to search
            if node.leaf:

                # If the node is a leaf and the key is not present, then the key doesn't exist
                return False

            # Recursively delete the key from the appropriate child
            self._delete_from_child(node, key, keyIndex)

    def _delete_key_from_node(self, node, key, i):
        # Case 1: Key is present in a node and is a leaf node just remove it directly
        if node.leaf:
            #print("YESSSSSSSSSSSSSSSS")
            del node.keys[i]
        else:
            # Case 2: Key is present in a node and is an internal node
            # Replace the key with its predecessor (max value from the left child) and recursively delete the predecessor
            predecessor = self._get_predecessor(node, i)
            node.keys[i] = predecessor
            # Recursively delete predecessor from subtree
            self._delete(node.children[i], predecessor)
            if key in node.keys:
              self._delete_key_from_node(node, key, node.keys.index(key))
            else:
              #key is deleted
              return
    def _delete_from_child(self, parentNode, keytoDelete, childIndex):
        #print("Delete from child")
        # Case 3: Key is not present in a node, and the child has the minimum number of keys
        if len(parentNode.children[childIndex].keys) == self.t - 1:
            # Perform necessary rotations or merges to ensure that the child has enough keys
            self._fix_deficiency(parentNode, childIndex)

        # Recursively delete the key from the appropriate child
        self._delete(parentNode.children[childIndex], keytoDelete)

    def _get_predecessor(self, parentNode, childIndex):
        # Helper function to find the predecessor (max value) in the subtree rooted at children[i]
        current_node = parentNode.children[childIndex]
        while not current_node.leaf:
            current_node = current_node.children[-1]
        # Return the maximum value which is the predecessor
        return current_node.keys[-1]
    def _fix_deficiency(self, parentNode, childIndex):
      #print("DEFICI")
      # Case 1: Borrow a key from the left sibling
      if childIndex > 0 and len(parentNode.children[childIndex - 1].keys) > self.t - 1:
          #print("b_left")
          self._borrow_from_left_sibling(parentNode, childIndex)

      # Case 2: Borrow a key from the right sibling
      elif childIndex < len(parentNode.children) - 1 and len(parentNode.children[childIndex + 1].keys) > self.t - 1:
          #print("b_right")
          self._borrow_from_right_sibling(parentNode, childIndex)

      # Case 3: Merge with the left sibling which is not required as merging with left node can cause culstering and inaccuracy in btree
      elif childIndex > 0:
          #print("m_left")
          #print(parentNode.keys,parentNode.children, childIndex)
          #self._merge_with_left_sibling(parentNode, childIndex)
          pass

      # Case 4: Merge with the right sibling
      elif len(parentNode.children)==0:
          #print("m_right")
          self._merge_with_right_sibling(parentNode, childIndex)
      #Case 5: if there is no deficiency do not have do any changes
      else:
        return

    def _borrow_from_left_sibling(self, parentNode, childIndex):
        # Borrow a key from the left sibling and adjust the keys in the nodes
        child = parentNode.children[childIndex]
        left_sibling = parentNode.children[childIndex - 1]

        # Move the key from the left sibling to the parent
        child.keys.insert(0, parentNode.keys[childIndex - 1])
        parentNode.keys[childIndex - 1] = left_sibling.keys.pop()

        # If the child is not a leaf, move the child pointer from the left sibling to the child
        if not child.leaf:
            child.children.insert(0, left_sibling.children.pop())

    def _borrow_from_right_sibling(self, parentNode, childIndex):
        # Borrow a key from the right sibling and adjust the keys in the nodes
        child = parentNode.children[childIndex]
        right_sibling = parentNode.children[childIndex + 1]

        # Move the key from the right sibling to the parent
        child.keys.append(parentNode.keys[childIndex])
        parentNode.keys[childIndex] = right_sibling.keys.pop(0)
        #parentNode.keys=parentNode.keys[childIndex].pop()

        # If the child is not a leaf, move the child pointer from the right sibling to the child
        if not child.leaf:
            child.children.append(right_sibling.children.pop(0))

    def _merge_with_left_sibling(self, parentNode, childIndex):
        # Merge the child with its left sibling, adjusting keys and pointers
        if len(parentNode.keys)>=1:
          child = parentNode.children[childIndex]
          left_sibling = parentNode.children[childIndex - 1]

          # Move the key from the parent to the left sibling
          left_sibling.keys.append(parentNode.keys.pop(childIndex - 1))

          # Move all keys and pointers from the child to the left sibling
          left_sibling.keys.extend(child.keys)
          left_sibling.children.extend(child.children)

          # Remove the child from the parent's keys and children lists
          if len(parentNode.keys)>0:
            parentNode.keys.pop(childIndex - 1)
            parentNode.children.pop(childIndex)
        else:
          return

    def _merge_with_right_sibling(self, parentNode, childIndex):
        # Merge the child with its right sibling, adjusting keys and pointers
        child = parentNode.children[childIndex]
        right_sibling = parentNode.children[childIndex + 1]

        # Move the key from the parent to the child
        child.keys.append(parentNode.keys.pop(childIndex))

        # Move all keys and pointers from the right sibling to the child
        child.keys.extend(right_sibling.keys)
        child.children.extend(right_sibling.children)

        # Remove the right sibling from the parent's keys and children lists
        if len(parentNode.keys)>0:
          parentNode.keys.pop(childIndex)
          parentNode.children.pop(childIndex + 1)
    def pre_order_traversal(self):
        if self.root:
            self._pre_order_traversal(self.root)

    def _pre_order_traversal(self, node):
        if node:
            # Visit the current node
            print(node.keys, end=" ")

            # Recursively traverse each child
            for child in node.children:
                self._pre_order_traversal(child)
    def post_order_traversal(self):
        if self.root:
            self._post_order_traversal(self.root)

    def _post_order_traversal(self, node):
        if node:
            # Recursively traverse each child
            for child in node.children:
                self._post_order_traversal(child)

            # Visit the keys
            print(node.keys, end=" ")
    def in_order_traversal(self):
        if self.root:
            self._in_order_traversal(self.root)

    def _in_order_traversal(self, node):
        if node:
            # Traverse each child and its associated key
            for i in range(len(node.keys)):
                # Recursively traverse the left child
                if i < len(node.children):
                  self._in_order_traversal(node.children[i])

                # Print the key
                print(node.keys[i], end=" ")

            # Traverse the rightmost child
            if len(node.children) > len(node.keys):
              self._in_order_traversal(node.children[-1])

    def bfs(self):
      if self.root:
            self._bfs(self.root)
    def _bfs(self,node):
        if not self.root:
            return
        # Use a queue for level-order traversal
        queue = deque([self.root])
        while queue:
            current_level_size = len(queue)

            for _ in range(current_level_size):
                node = queue.popleft()

                # Print keys in the current node
                print(node.keys, end=" ")

                # Enqueue child nodes if they exist
                if not node.leaf:
                    queue.extend(node.children)

    def print_by_levels(self):
        if not self.root:
            print("B-tree is empty")
            return

        # Use a queue for level-order traversal
        queue = deque([self.root])

        while queue:
            current_level_size = len(queue)

            for _ in range(current_level_size):
                node = queue.popleft()

                # Print keys in the current node
                print(node.keys, end=" ")

                # Enqueue child nodes if they exist
                if not node.leaf:
                    queue.extend(node.children)

            print()  # Move to the next line for the next level


# Data File Parsing
### Please Upload the file address in this part of the code : )
Example: file("Address_to_the_file")

In [ ]:
#make sure to write the exact file destination address to this function to run.
def file(file_name):
  import pandas as pd
  import csv
  #fining the file format based on the last 3 characters after the period and based on the file format writing a read function
  file_name_split = file_name.split('.')[-1].lower()
  if file_name_split == 'csv':
      df=pd.read_csv(file_name)
  elif file_name_split ==' xlsx':
      df=pd.read_excel(file_name)
  else:
      return 'Unknown File Format'
  # df=pd.read_csv(file_name)
  # Create a DataFrame using the extracted data and prinitng the first 2 rows for reference
  df = pd.DataFrame(df)
  print("here is a the sample DF for reference")
  print(df.head(2))
  #since the id's are usually present in the first column making it default to zero without bothering the user with more questions :)
  #index_ask=int(input("Enter the index(startting with 0) of the column with the ID's or nodes: "))
  index_ask=0
  #asking for min degree
  min_degree=int(input("What's the min degree (t) you would like to be (2/3/4/5)?: "))
  btree=BTree(min_degree)
  #iteratively looping through the first column of the index and inserting the elements to the btree
  for x in df.iloc[0:,index_ask]:
    #print(x, "-> inserted")
    btree.insert(x)
  status=True
  #priniting the tree by levels
  btree.print_by_levels()
  #asking user for that operation would they like to do
  '''
  1. Search
  2. Remove
  3. Traversals
    a. DFS
      i. Pre Order
      ii. In Order
      iii. Post Order
    b. BFS (Level Order)
  4. Exit
  '''
  while status==True:
    user_input=input("Would you like to search(s)/remove(r)/DFS, BFS traversals(t)/exit(e): ")
    user_input.lower()
    if user_input=="t" or user_input=="traversals" or user_input=="traversal":
      user_input2=str(input("Preorder(1)/InOrder(2)/PostOrder(3)/Level Order(BFS)(4): "))
      if user_input2=='1':
        print("Root-Left Subtree- Right Subtree")
        btree.pre_order_traversal()
      if user_input2=='2':
        print("Left Subtree -Root- Right Subtree")
        btree.in_order_traversal()
      if user_input2=='3':
        print("Left Subtree - Right Subtree- Root")
        print(btree.post_order_traversal())
      if user_input2=='4' or user_input2=="bfs" or user_input2=="BFS":
        print("Level Order Traversal")
        print(btree.bfs())
    if user_input =="s" or user_input=="search":
      s_element=int(input("What element would you like to search for?: "))
      s_bool=btree.search(s_element)
      print(s_bool)
      if s_bool== True:
        #print(df.iloc[s_element,:])
        filtered_df = df[df[df.columns[index_ask]] == s_element]
        print(filtered_df)
      else:
        pass
    if user_input=="r" or user_input=="remove":
      r_element=int(input("What element would you like to remove?: "))
      btree.delete(r_element)
      r_df=df[df[df.columns[index_ask]] == r_element]
      df = df[~df[df.columns[index_ask]].isin(r_df[df.columns[index_ask]])]
      print("The Updated Tree is: ")
      btree.print_by_levels()
      #print(df)
    if user_input=="e" or user_input=="exit":
      print("Here is the Final BTree")
      btree.print_by_levels()
      status=False

# write the address to the exact datafile
file("Iris.csv")


here is a the sample DF for reference
   Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
0   1            5.1           3.5            1.4           0.2  Iris-setosa
1   2            4.9           3.0            1.4           0.2  Iris-setosa
What's the min degree (t) you would like to be (2/3/4/5)?: 3
[27, 54, 81, 108] 
[9, 18] [36, 45] [63, 72] [90, 99] [117, 126, 135] 
[3, 6] [12, 15] [21, 24] [30, 33] [39, 42] [48, 51] [57, 60] [66, 69] [75, 78] [84, 87] [93, 96] [102, 105] [111, 114] [120, 123] [129, 132] [138, 141, 144, 147] 
[1, 2] [4, 5] [7, 8] [10, 11] [13, 14] [16, 17] [19, 20] [22, 23] [25, 26] [28, 29] [31, 32] [34, 35] [37, 38] [40, 41] [43, 44] [46, 47] [49, 50] [52, 53] [55, 56] [58, 59] [61, 62] [64, 65] [67, 68] [70, 71] [73, 74] [76, 77] [79, 80] [82, 83] [85, 86] [88, 89] [91, 92] [94, 95] [97, 98] [100, 101] [103, 104] [106, 107] [109, 110] [112, 113] [115, 116] [118, 119] [121, 122] [124, 125] [127, 128] [130, 131] [133, 134] [136, 137] [1